<a href="https://colab.research.google.com/github/hwarang97/Image_classification/blob/main/lenet_5_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from torch.nn.modules.activation import Softmax
import torch
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

import torch.nn.functional as F

In [14]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor() # 이미지, 배열을 텐서로 변경하고, 픽셀값을 정규하시켜줌
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor() 
)

train_dataloader = DataLoader(training_data, batch_size=1)
test_dataloader = DataLoader(test_data, batch_size=1)

class Lenet_5(nn.Module):
  # def __init__(self):
  #   super(Lenet_5, self).__init__()
  #   self.flatten = nn.Flatten()
  #   self.conv_sigmoid_stack = nn.Sequential(
  #       nn.Conv2d(1, 6, 5, padding=4), # input 1*32*32, output 6*28*28, same padding
  #       nn.Sigmoid(),
  #       nn.AvgPool2d(2),
  #       nn.Conv2d(6, 16, 5),
  #       nn.Sigmoid(),
  #       nn.AvgPool2d(2),
  #   )

  #   self.linear_sigmoid_stack = nn.Sequential(
  #     nn.Linear(16*5*5, 84),
  #     nn.Sigmoid(),
  #     nn.Linear(84, 10)
  #   )

  # def forward(self, x):
  #   x1 = self.conv_sigmoid_stack(x)
  #   x2 = self.flatten(x1)
  #   logits = self.linear_sigmoid_stack(x2)
  #   return logits

  def __init__(self):
        super(Lenet_5, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5은 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
      x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
      # 크기가 제곱수라면, 하나의 숫자만을 특정(specify)
      x = F.max_pool2d(F.relu(self.conv2(x)), 2)
      x = torch.flatten(x, 1) # 배치 차원을 제외한 모든 차원을 하나로 평탄화(flatten)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader, 1):

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Lenet_5().to(device)
print(model)

Lenet_5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
learning_rate = 1e-3
batch_size = 1
epochs = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
  print(f'Epoch {t+1}\n-----------------------------------')
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print('Done!!!')

Epoch 1
-----------------------------------


RuntimeError: ignored